In [20]:
"""
This notebook implemenets the CDF based histogram equalization 
"""

'\nThis notebook implemenets the CDF based histogram equalization \n'

In [21]:
import cv2 as cv;
import numpy as np;

In [22]:
probabilties = {}

In [23]:
def show(prompt,img):
    cv.namedWindow(prompt, cv.WINDOW_NORMAL)
    cv.imshow(prompt, img)
    cv.waitKey(0) 
    cv.destroyAllWindows()


In [24]:
cdfDict = {};

In [25]:
img = cv.imread("wiki.jpg");


In [26]:
imgGray = cv.cvtColor(img,cv.COLOR_RGB2GRAY)

In [27]:
img  = imgGray
def probability(value):
    count =   np.count_nonzero(img == value)/(img.shape[0]*img.shape[1])
    # setting value in dictionary for future use
    probabilties[value] = count;
    # print(f"prob({value }) =  {count}")

def getProbabilty(value):

    if(value not in probabilties.keys()):
        return probability(value);
    return probabilties[value]

def getCommulativeProbabilty(value):
    
    valueArr = list(probabilties.values());
    keys = list(probabilties.keys());

    idx = keys.index(value);
    cdfVal = 0;
    for i in range(idx+1):
        cdfVal += valueArr[i];
    
    cdfDict[value] = cdfVal;
    return cdfVal;



In [28]:
vGetProbabilty = np.vectorize(getProbabilty);
vGetCdf = np.vectorize(getCommulativeProbabilty)

In [29]:
vGetProbabilty(imgGray)

array([[0.056212933394160586, None, None, ..., 0.02597513686131387,
        0.025932367700729927, 0.0028370209854014596],
       [0.03190579379562044, 0.025875342153284672, 0.04546361770072993,
        ..., 0.023594320255474453, 0.015482436131386862,
        0.001739279197080292],
       [0.025918111313868612, 0.025533188868613138, 0.025918111313868612,
        ..., 0.01285926094890511, 0.006757527372262774,
        0.002295278284671533],
       ...,
       [0.01474110401459854, 0.012089416058394161, 0.012089416058394161,
        ..., 0.028384466240875914, 0.02531934306569343,
        0.0024520985401459854],
       [0.007669936131386861, 0.009623061131386862, 0.012089416058394161,
        ..., 0.025875342153284672, 0.023123859489051095,
        0.0023237910583941606],
       [0.006386861313868613, 0.007669936131386861, 0.01474110401459854,
        ..., 0.024905907846715328, 0.025932367700729927,
        0.001967381386861314]], dtype=object)

In [30]:
vGetCdf(imgGray)

array([[0.05621293, 0.10167655, 0.12876369, ..., 0.15473882, 0.49897354,
        0.68487682],
       [0.57962192, 0.74300011, 0.10167655, ..., 0.47304117, 0.40662067,
        0.94234717],
       [0.18065693, 0.30210709, 0.18065693, ..., 0.39113823, 0.69163435,
        0.94998859],
       ...,
       [0.98266423, 0.65780395, 0.65780395, ..., 0.64571453, 0.32742644,
        0.6820398 ],
       [0.96404539, 0.94060789, 0.65780395, ..., 0.74300011, 0.3505503 ,
        0.87110801],
       [0.95637546, 0.96404539, 0.98266423, ..., 0.81823107, 0.49897354,
        0.86878422]])

In [31]:
# finding a cdf for every element
cdfValues = cdfDict.values()



In [32]:
def scale(pixel):
    return pixel*255

vScale = np.vectorize(scale)

In [33]:
## scaled each value in the image by 255
scaledValues = vScale(list(cdfValues))

lookUpTable = dict(zip(np.array(list(cdfDict.keys())), np.array(scaledValues) ))


In [34]:
def addValuesFromLUT(pixel):
    """function that replaces the values in the image with the values found in scaled cdf"""
    return lookUpTable[pixel]

vLookUpTable = np.vectorize(addValuesFromLUT)

In [35]:
equalized = vLookUpTable(img).astype(np.uint8)

In [36]:
print(equalized)

[[ 14  25  32 ...  39 127 174]
 [147 189  25 ... 120 103 240]
 [ 46  77  46 ...  99 176 242]
 ...
 [250 167 167 ... 164  83 173]
 [245 239 167 ... 189  89 222]
 [243 245 250 ... 208 127 221]]


In [37]:
show("histogram wualized ",equalized)
show("original ",img)

In [19]:
cv.imwrite("enh_wiki_by_cdf.jpg",equalized)

True

In [161]:
byCv = cv.equalizeHist(img)
show("original ",img)
show("by cv ",byCv)
show("by us ",equalized)